# Discrete Solution Methods: Single vs Multi Index Methods

We compare single and multi index methods for computing the wealth consumption ratio.

## Single Index Code

In these models, the state space is a multi-dimensional grid.  Single index code stacks this multi-dimensional grid into a single array, with a single index.  Transition probabilities across states can then be represented by a Markov matrix.

The code is neat but the size of the Markov matrix is $n \times n$ when $n$ is the size of the state space.  Similarly, the Jacobian for Newton's method is $n \times n$. This matrix cannot be instantiated or manipulated when $n$ is large.  Hence the single index code fails for large $n$.

Here's a test of the code:

In [1]:
run ../src/wc_ratio_single_index.py

Let's start with $n=3^4$, just to verify that the code works.

In [2]:
s = 3
ssy = SSY(L=s, K=s, I=s, J=s, build_single_index=True)

In [3]:
print(solvers.keys())

dict_keys(['newton', 'anderson', 'successive_approx'])


In [4]:
%%time
w_star_single = wc_ratio_single_index(ssy, algorithm="newton")

Beginning iteration


iter = 0, error = 4595.925224178379
iter = 1, error = 4419.779393640898
iter = 2, error = 23.27635205848435
iter = 3, error = 0.05675964976137493
iter = 4, error = 2.7144183150085155e-06
iter = 5, error = 3.512923285597935e-11
Iteration converged after 6 iterations
CPU times: user 6.51 s, sys: 236 ms, total: 6.75 s
Wall time: 5.34 s


In [5]:
w_star_single

array([[[[829.19974818, 852.09611779, 874.8311396 ],
         [807.26402064, 840.28614206, 873.68953344],
         [775.83240529, 823.10889033, 872.57959972]],

        [[824.39045411, 847.14345628, 869.7361625 ],
         [802.594611  , 835.41276679, 868.60968046],
         [771.35976012, 818.34891268, 867.51834482]],

        [[813.53342141, 835.96920036, 858.24702393],
         [792.0447625 , 824.40939541, 857.14737607],
         [761.24501913, 807.59180431, 856.08817106]]],


       [[[866.45033863, 890.39607943, 914.17288536],
         [843.50839069, 878.04197506, 912.97419386],
         [810.63717679, 860.07325395, 911.80428301]],

        [[861.42023005, 885.21601371, 908.84396289],
         [838.62459721, 872.9448429 , 907.66109541],
         [805.95919075, 855.09473527, 906.51064566]],

        [[850.06602521, 873.53001383, 896.82861674],
         [827.59169163, 861.43759001, 895.67383664],
         [795.38137581, 843.84507463, 894.55703017]]],


       [[[905.3945672 , 930.43

When $n$ gets to around 12, the kernel dies or the program fails with an out of memory error.

## Multi-Index Code

With multi-index code, the state remains as a multidimensional grid and the Markov matrix of transitions does not need to be instantiated.  Transition probabilities are calculated at run time.

In addition, we use some JAX tricks to avoid instantiating the Jacobian used in Newton iteration.

To put this on the GPU via JAX, we use broadcasting to define the necessary calculations.

In [1]:
run ../src/wc_ratio_multi_index.py

Here's a quick test at $n=3^4$:

In [2]:
s = 3
ssy = SSY(L=s, K=s, I=s, J=s, build_single_index=False)

In [ ]:
%%time
w_star_multi_newton = wc_ratio_multi_index(ssy, algorithm="newton")

Then numbers are very close to what we get with a single index calculation:

In [9]:
w_star_multi_newton

DeviceArray([[[[829.82993073, 852.74401371, 875.4966214 ],
               [807.8772417 , 840.92489178, 874.35410499],
               [776.42133489, 823.7343184 , 873.24322697]],

              [[825.01691459, 847.78751888, 870.39770055],
               [803.20421857, 836.0477448 , 869.27032013],
               [771.9452289 , 818.97065717, 868.17805489]],

              [[814.15148897, 836.60462393, 858.8996787 ],
               [792.64621538, 825.035867  , 857.7991534 ],
               [761.82267063, 808.2052336 , 856.73904419]]],


             [[[865.41862459, 889.33525697, 913.08316573],
               [842.50461325, 876.99628775, 911.88611528],
               [809.67334179, 859.04956287, 910.71794249]],

              [[860.39464688, 884.16150486, 907.76073825],
               [837.62677231, 871.90536807, 906.57949248],
               [805.00105749, 854.07711212, 905.43075702]],

              [[849.05423229, 872.48969925, 895.75998741],
               [826.60726546, 860.41209128, 

Now let's try with large $n$ and compare.

In [3]:
s = 18
ssy = SSY(L=s, K=s, I=s, J=s, build_single_index=False)

The size of the state space is 

In [4]:
s**4

104976

In [12]:
%%time
w_star_multi_sa = wc_ratio_multi_index(ssy, algorithm="successive_approx")

Beginning iteration


iter = 0, error = 20.69205479122877
iter = 1000, error = 0.04128829140847756
iter = 2000, error = 4.7093378157114785e-05
Iteration converged after 2907 iterations
CPU times: user 25min 29s, sys: 2.62 s, total: 25min 32s
Wall time: 25min 25s


In [ ]:
%%time
w_star_multi_newton = wc_ratio_multi_index(ssy, algorithm="newton")

In [14]:
np.max(np.abs(w_star_multi_newton - w_star_multi_sa))

[autoreload of utils failed: Traceback (most recent call last):
  File "/home/john/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/john/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/john/anaconda3/lib/python3.9/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'utils'
]


1.4647637726739049e-05